# Cross Validation

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    stream=sys.stdout,
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
from omegaconf import OmegaConf

c = OmegaConf.load("../config/main.yaml")

c.settings.debug = False
c.wandb.enabled = False
c.wandb.group = "LB"
c.wandb.dir = "../../cache/"
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

pretraind_dir = "../../datasets/trainings"

In [4]:
pretrained = f"""

- dir: {pretraind_dir}/2022-03-18_10-49-32/fold0/ 
  model: tabnet
  n_class: 1

- dir: {pretraind_dir}/2022-03-13_20-45-32/fold0/ 
  model: lightgbm
  n_class: 1

- dir: {pretraind_dir}/2022-03-11_20-25-09/fold1/
  model: ump_1
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-11_20-25-15/fold2/
  model: ump_1
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-11_20-26-13/fold4/
  model: ump_1
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-11_20-26-19/fold5/
  model: ump_1
  model_input: 300
  n_class: 1

- dir: {pretraind_dir}/2022-03-12_21-58-26/fold1/
  model: ump_1dcnn
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-12_21-58-32/fold2/
  model: ump_1dcnn
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-12_21-59-36/fold4/
  model: ump_1dcnn
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-12_21-59-41/fold5/
  model: ump_1dcnn
  model_input: 300
  n_class: 1

- dir: {pretraind_dir}/2022-03-17_20-47-04/fold1/
  model: ump_lstm
  model_input: 300
  n_class: 10
- dir: {pretraind_dir}/2022-03-17_20-47-09/fold2/
  model: ump_lstm
  model_input: 300
  n_class: 10
- dir: {pretraind_dir}/2022-03-17_20-47-41/fold4/
  model: ump_lstm
  model_input: 300
  n_class: 10
- dir: {pretraind_dir}/2022-03-17_20-47-47/fold5/
  model: ump_lstm
  model_input: 300
  n_class: 10

"""

_pretrained = f"""

- dir: {pretraind_dir}/2022-03-11_20-25-04/fold0/
  model: ump_1
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-11_20-26-08/fold3/
  model: ump_1
  model_input: 300
  n_class: 1

- dir: {pretraind_dir}/2022-03-12_21-58-21/fold0/
  model: ump_1dcnn
  model_input: 300
  n_class: 1
- dir: {pretraind_dir}/2022-03-12_21-59-31/fold3/
  model: ump_1dcnn
  model_input: 300
  n_class: 1

- dir: {pretraind_dir}/2022-03-17_20-46-59/fold0/
  model: ump_lstm
  model_input: 300
  n_class: 10
- dir: {pretraind_dir}/2022-03-17_20-47-36/fold3/
  model: ump_lstm
  model_input: 300
  n_class: 10


"""

c.params.pretrained = OmegaConf.create(pretrained)

In [5]:
log.info(OmegaConf.to_yaml(c))

2022-03-18 19:57:22,438 [INFO] [3244290467] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: true
  entity: imokuri
  project: ump
  dir: ../../cache/
  group: LB
settings:
  print_freq: 100
  gpus: '6'
  dirs:
    working: ..
    input: ../../inputs/
    input_minimal: ${hydra:runtime.cwd}/../datasets/inputs/
    feature: ${settings.dirs.input}features/
    preprocess: ${settings.dirs.input}preprocess/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
  training_method: nn
params:
  seed: 7440
  n_class: 1
  preprocess: []
  pca_n_components: 50
  n_fold: 5
  skip_training: false
  epoch: 10
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: combinational_purged
  group_name: investment_id
  ti

In [6]:
import os

import pandas as pd
import src.utils as utils
from src.get_score import record_result
from tqdm.notebook import tqdm

In [7]:
run = utils.setup_wandb(c)

2022-03-18 19:57:24,312 [ERROR] [jupyter] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: imokuri (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2022-03-18 19:57:30,840 [INFO] [utils] WandB initialized. name: wild-vortex-412, id: 29v0xvi9


In [8]:
train = pd.read_feather("../../inputs/train.f")
train = train.loc[:, ["row_id", "time_id", "target"]]
train.set_index("row_id", inplace=True)

In [9]:
preds_col = []

# 各 OOF の結果を読み取り
for n, training in tqdm(enumerate(c.params.pretrained), total=len(c.params.pretrained)):
    preds_col.append(f"preds{n}")
    oof_df = pd.read_feather(os.path.join(training.dir.rsplit("/", 2)[0], "oof_df.f")).set_index("row_id")

    if training.model == "lightgbm":
        train[f"preds{n}"] = oof_df["preds"].groupby("row_id").sum()
    else:
        train[f"preds{n}"] = oof_df["preds"]

# 各行の OOF の結果の数をカウントする
train["count_oof"] = len(c.params.pretrained) - train.isnull().sum(axis=1)  # + 4

# OOF の結果がない行を 0 埋め
train.fillna(0, inplace=True)

# OOF の結果をマージ
train["preds"] = 0
for col in preds_col:
    train["preds"] += train[col]

# 推論結果がないものは除外
train = train[train["preds"] != 0.0]

# 複数 OOF を加算している場合は、OOFの数で割る
train["preds"] = train["preds"] / train["count_oof"]

  0%|          | 0/14 [00:00<?, ?it/s]

In [10]:
train

,time_id,target,preds0,preds1,preds2,preds3,preds4,preds5,preds6,preds7,preds8,preds9,preds10,preds11,preds12,preds13,count_oof,preds
row_id,,,,,,,,,,,,,,,,,,
0_1,0,-0.300875,0.012218,0.096153,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,2,0.054185
0_2,0,-0.231040,-0.054585,-0.065383,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,2,-0.059984
0_6,0,0.568807,-0.022041,0.031596,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,2,0.004777
0_7,0,-1.064780,-0.145309,-0.048442,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,2,-0.096876
0_8,0,-0.531940,-0.050417,-0.061919,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,2,-0.056168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219_3768,1219,0.033600,-0.027354,-0.089907,0.0,0.0,0.0,-0.048523,0.0,0.0,0.0,-0.069946,0.0,0.0,0.0,-0.123535,5,-0.071853
1219_3769,1219,-0.223264,-0.029044,-0.062147,0.0,0.0,0.0,-0.055573,0.0,0.0,0.0,-0.062683,0.0,0.0,0.0,-0.137329,5,-0.069355
1219_3770,1219,-0.559415,0.013071,0.020162,0.0,0.0,0.0,0.071899,0.0,0.0,0.0,0.043213,0.0,0.0,0.0,0.066040,5,0.042877


In [11]:
train["count_oof"].value_counts()

5    2094272
2    1047138
Name: count_oof, dtype: int64

In [12]:
train = train[train["count_oof"] != 1]
train["count_oof"].value_counts()

5    2094272
2    1047138
Name: count_oof, dtype: int64

In [13]:
record_result(c, train, c.params.n_fold)

2022-03-18 19:58:02,376 [INFO] [get_score] Score: 0.14552


0.14552131359296958